In [6]:
import pandas as pd
import numpy as np
import os
import glob
from glob import glob
import re
import luigi
from operator import itemgetter
from persiantools import characters
from persiantools.jdatetime import JalaliDate
import persiantools
import datetime
from re import search
import itertools
from statistics import mode
import datetime as dt
from joblib import Parallel, delayed
import chardet
import requests
from bs4 import BeautifulSoup
import json
import time
import io
import jdatetime
# import TSETMCScraping
import ar_to_fa
import clean_cols
import str_to_date
import str_to_jdate
import str_to_time

In [12]:
class TSETMC_Sraping(luigi.Task):
    read_path= luigi.Parameter(default=None)
    dirr_path= luigi.Parameter(default=None)
    def requiers(self):
        return None
    def output(self):
        return luigi.LocalTarget(self.dirr_path)
    def run(self):
        if os.path.isfile(self.dirr_path):
            if self.read_path ==None:
                pass
            else:
                read_link= pd.read_excel(self.read_path)
                df= TSETMCScraping.stock_information(read_link)
                df.to_csv(self.output().path, mode='a',index=False, header=None)
        else:
            read_link= pd.read_excel(self.read_path)
            df.to_csv(self.output().path, index=False,encoding='utf-8-sig')


In [14]:
# path_1=input('Insert the read path: ')
# path_2= input('insert the output path: ')
class cleaning_trade_history_pandas (luigi.Task):
    read_path= luigi.Parameter(default=None)
    dirr_path= luigi.Parameter(default=None)
    def requiers(self):
        return TSETMC_Sraping()
    def output(self):
        return luigi.LocalTarget(self.dirr_path)
    def run(self):
        df= pd.read_csv(self.read_path)
        clean_cols(df)
        ar_to_fa(df)
        str_to_date(df)
        str_to_jdate(df)
        str_to_time(df)
# if __name__=='__main__':
#     luigi.build([cleaning_trade_history_pandas()],read_path= path_1,dirr_path=path_2,local_scheduler=True)

NameError: name 'self' is not defined

In [6]:
class HellowWorld(luigi.Task):
    def requires(self):
        return None
    def output(self):
        return luigi.LocalTarget('helloworld.txt')
    def run (self):
        with self.output().open('w') as outfile:
            outfile.write('Hello world\n')
if __name__ == '__main__':
    luigi.build([HellowWorld()], local_scheduler=True)

DEBUG: Checking if HellowWorld() is complete
INFO: Informed scheduler that task   HellowWorld__99914b932b   has status   PENDING
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 1
INFO: [pid 48680] Worker Worker(salt=034725087, workers=1, host=POOLBANK-RDS2, username=ah.roudsaz, pid=48680) running   HellowWorld()
INFO: [pid 48680] Worker Worker(salt=034725087, workers=1, host=POOLBANK-RDS2, username=ah.roudsaz, pid=48680) done      HellowWorld()
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   HellowWorld__99914b932b   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Done
DEBUG: There are no more tasks to run at this time
INFO: Worker Worker(salt=034725087, workers=1, host=POOLBANK-RDS2, username=ah.roudsaz, pid=48680) was stopped. Shutting down Keep-Alive thread
INFO: 
===== Luigi Execution Summary =====

Scheduled 1 tasks of which:
* 1 ran successfully:
 